In [2]:
# ==============================================================================
# VERSION: Robust & Leak-Free Model
# This version implements a leak-proof cross-validation scheme for reliable
# hyperparameter tuning and trains a powerful final model.
# ==============================================================================
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import optuna
import warnings

warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# --- Define File Paths ---
TRAIN_FILE_PATH = '/Users/shrinarayan/Desktop/Prediction-PurchaseValues/dataset/train_data.csv'
TEST_FILE_PATH = '/Users/shrinarayan/Desktop/Prediction-PurchaseValues/dataset/test_data.csv'

# ==============================================================================
# PART 1: FEATURE ENGINEERING & PREPROCESSING CLASSES (Largely Unchanged)
# ==============================================================================

class AdvancedFeatureEngineering(BaseEstimator, TransformerMixin):
    """Creates time-based and interaction features from session-level data."""
    def fit(self, X, y=None): return self
    def transform(self, X):
        X_copy = X.copy()
        X_copy['date'] = pd.to_datetime(X_copy['date'], format='%Y%m%d')
        X_copy['sessionYear'] = X_copy['date'].dt.year
        X_copy['sessionMonth'] = X_copy['date'].dt.month
        X_copy['sessionDayOfWeek'] = X_copy['date'].dt.dayofweek
        X_copy['sessionHour'] = pd.to_datetime(X_copy['sessionStart'], unit='s').dt.hour
        X_copy['is_weekend'] = (X_copy['sessionDayOfWeek'] >= 5).astype(int)
        X_copy['month_day_interaction'] = X_copy['sessionMonth'].astype(str) + '_' + X_copy['sessionDayOfWeek'].astype(str)
        X_copy['browser_os_interaction'] = X_copy['browser'].astype(str) + '_' + X_copy['os'].astype(str)
        X_copy['geo_channel_interaction'] = X_copy['geoNetwork.continent'].astype(str) + '_' + X_copy['userChannel'].astype(str)
        X_copy['device_channel_interaction'] = X_copy['deviceType'].astype(str) + '_' + X_copy['userChannel'].astype(str)
        X_copy['hits_per_pageview'] = X_copy['totalHits'] / (X_copy['pageViews'].replace(0, 1)) # Avoid division by zero
        X_copy['ad_page_binned'] = X_copy['trafficSource.adwordsClickInfo.page'].apply(lambda p: 1 if p == 1.0 else (2 if pd.notna(p) else 0))
        cols_to_drop = ['date', 'sessionStart', 'sessionId', 'trafficSource.adwordsClickInfo.page']
        X_copy = X_copy.drop(columns=cols_to_drop, errors='ignore')
        return X_copy

class TargetEncoder(BaseEstimator, TransformerMixin):
    """Target encodes categorical features with smoothing."""
    def __init__(self, columns=None, smoothing=10):
        self.columns, self.smoothing = columns, smoothing
        self.mappings_, self.global_mean_ = {}, 0
    def fit(self, X, y):
        X_fit, y_fit = X.copy(), y.copy()
        self.global_mean_ = np.mean(y_fit)
        for col in self.columns:
            X_fit[col] = X_fit[col].fillna('missing').astype(str)
            agg = y_fit.groupby(X_fit[col]).agg(['mean', 'count'])
            smooth_mean = (agg['count'] * agg['mean'] + self.smoothing * self.global_mean_) / (agg['count'] + self.smoothing)
            self.mappings_[col] = smooth_mean.to_dict()
        return self
    def transform(self, X):
        X_transform = X.copy()
        for col in self.columns:
            X_transform[col] = X_transform[col].fillna('missing').astype(str)
            X_transform[col] = X_transform[col].map(self.mappings_[col]).fillna(self.global_mean_)
        return X_transform

def create_user_aggregates(df):
    """Creates user-level aggregates. Designed to be used on a training fold."""
    agg = df.groupby('userId').agg(
        user_session_count=('sessionId', 'nunique'),
        user_total_hits=('totalHits', 'sum'),
        user_avg_hits=('totalHits', 'mean'),
        user_total_pageviews=('pageViews', 'sum'),
        user_avg_pageviews=('pageViews', 'mean'),
        user_purchase_count=('made_purchase', 'sum'),
        user_total_purchase_value=('purchaseValue', 'sum'),
    ).reset_index()
    agg['user_conversion_rate'] = agg['user_purchase_count'] / agg['user_session_count']
    agg['user_avg_purchase_value'] = agg['user_total_purchase_value'] / (agg['user_purchase_count'] + 1e-6)
    return agg

# ==============================================================================
# PART 2: DATA LOADING AND INITIAL PREPARATION
# ==============================================================================

print("Loading and preparing data...")
df_train = pd.read_csv(TRAIN_FILE_PATH, dtype={'fullVisitorId': 'str', 'sessionId': 'str'})
df_test = pd.read_csv(TEST_FILE_PATH, dtype={'fullVisitorId': 'str', 'sessionId': 'str'})

# Unify userId column name
df_train.rename(columns={'fullVisitorId': 'userId'}, inplace=True)
df_test.rename(columns={'fullVisitorId': 'userId'}, inplace=True)

# Drop constant columns
one_value_cols = [col for col in df_train.columns if df_train[col].nunique(dropna=False) == 1]
df_train = df_train.drop(columns=one_value_cols)
df_test = df_test.drop(columns=[c for c in one_value_cols if c in df_test.columns], errors='ignore')

# Prepare target variable and related columns
df_train['purchaseValue'] = df_train['purchaseValue'].fillna(0).astype(float)
df_train['made_purchase'] = (df_train['purchaseValue'] > 0).astype(int)
df_train['log_purchaseValue'] = np.log1p(df_train['purchaseValue'] / 1e6) # Scale before log

# Define features and target
X = df_train.drop(columns=['log_purchaseValue'])
y = df_train['log_purchaseValue']

# Get column names after initial feature engineering
temp_engineered_df = AdvancedFeatureEngineering().fit_transform(X)
user_level_numerical_cols = [
    'user_session_count', 'user_total_hits', 'user_avg_hits', 'user_total_pageviews',
    'user_avg_pageviews', 'user_purchase_count', 'user_total_purchase_value',
    'user_conversion_rate', 'user_avg_purchase_value'
]
session_level_numerical_cols = ['sessionNumber', 'pageViews', 'totalHits', 'hits_per_pageview']
numerical_cols = session_level_numerical_cols + user_level_numerical_cols
categorical_cols = [col for col in temp_engineered_df.columns if col not in numerical_cols and col != 'userId']
del temp_engineered_df # Clean up

print(f"\nIdentified {len(numerical_cols)} potential numerical features.")
print(f"Identified {len(categorical_cols)} potential categorical features.")

# ==============================================================================
# PART 3: ROBUST HYPERPARAMETER TUNING WITH LEAK-FREE CV
# ==============================================================================

# ==============================================================================
# PART 3: ROBUST HYPERPARAMETER TUNING (Corrected)
# ==============================================================================

def objective(trial, X_data, y_data):
    """Optuna objective function with a leak-free CV implementation."""
    params = {
        'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'random_state': 42, 'n_jobs': -1,
        'booster': 'gbtree',
        'n_estimators': trial.suggest_int('n_estimators', 500, 2000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05, log=True),
        'max_depth': trial.suggest_int('max_depth', 5, 12),
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
        
        # --- FIX APPLIED HERE ---
        'gamma': trial.suggest_float('gamma', 0, 5),                               # Removed log=True
        'lambda': trial.suggest_float('lambda', 1, 10, log=True),                 # L2 (lambda) can be log-scaled as its lower bound > 0
        'alpha': trial.suggest_float('alpha', 0, 10),                             # L1 (alpha) must be linear-scaled to include 0
    }

    cv = GroupKFold(n_splits=4)
    # ... rest of the function remains exactly the same ...
    # ...
    groups = X_data['userId']
    cv_scores = []

    # --- THE LEAK-FREE CROSS-VALIDATION LOOP ---
    for fold, (train_idx, val_idx) in enumerate(cv.split(X_data, y_data, groups)):
        print(f"  --- Fold {fold+1}/4 ---")
        X_train, X_val = X_data.iloc[train_idx], X_data.iloc[val_idx]
        y_train, y_val = y_data.iloc[train_idx], y_data.iloc[val_idx]

        # 1. Create user aggregates ONLY from the training fold's data
        # This is the crucial step to prevent data leakage.
        user_aggs_fold = create_user_aggregates(X_train)

        # 2. Apply advanced feature engineering
        afe = AdvancedFeatureEngineering()
        X_train_eng = afe.fit_transform(X_train)
        X_val_eng = afe.transform(X_val)

        # 3. Merge aggregates into the engineered train and validation folds
        X_train_final = pd.merge(X_train_eng, user_aggs_fold, on='userId', how='left').fillna(0)
        X_val_final = pd.merge(X_val_eng, user_aggs_fold, on='userId', how='left').fillna(0)
        
        # 4. Define and fit the full preprocessing and model pipeline on the fold
        # The TargetEncoder is now correctly fitted only on train data for each fold.
        preprocessor = ColumnTransformer([
            ('num', Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]), numerical_cols),
            ('cat', TargetEncoder(columns=categorical_cols, smoothing=trial.suggest_int('smoothing', 5, 20)), categorical_cols)
        ], remainder='drop', n_jobs=-1)
        
        model = Pipeline([
            ('preprocessor', preprocessor),
            ('regressor', xgb.XGBRegressor(**params))
        ])

        model.fit(X_train_final, y_train)
        preds = model.predict(X_val_final)
        
        # Clip predictions as log values shouldn't be negative
        preds[preds < 0] = 0
        
        rmse = np.sqrt(mean_squared_error(y_val, preds))
        cv_scores.append(rmse)
        print(f"    Fold {fold+1} RMSE: {rmse:.5f}")

    mean_rmse = np.mean(cv_scores)
    print(f"  Trial Mean RMSE: {mean_rmse:.5f}\n")
    return mean_rmse

# --- Run Optuna Study ---
N_TRIALS = 30 # For a serious competition, consider 50-100 trials
print(f"\n--- Tuning Regressor with Optuna ({N_TRIALS} trials) using LEAK-FREE CV ---")
study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X, y), n_trials=N_TRIALS)

print("\nBest trial:")
best_params_from_study = study.best_trial.params
print(f"  Value (RMSE): {study.best_value:.5f}")
print("  Params: ")
for key, value in best_params_from_study.items():
    print(f"    {key}: {value}")

# ==============================================================================
# PART 4: FINAL MODEL TRAINING AND SUBMISSION
# ==============================================================================

print("\n--- Training Final Model with Best Parameters on ALL Data ---")

# 1. Create user aggregates using the ENTIRE training dataset. This is correct
# for the final model as it provides the most historical data for test predictions.
final_user_aggregates = create_user_aggregates(X)

# 2. Engineer features for the full train and test sets
afe_final = AdvancedFeatureEngineering()
X_train_eng_final = afe_final.fit_transform(X)
X_test_eng_final = afe_final.transform(df_test)

# 3. Merge aggregates into both sets
X_train_final = pd.merge(X_train_eng_final, final_user_aggregates, on='userId', how='left').fillna(0)
X_test_final = pd.merge(X_test_eng_final, final_user_aggregates, on='userId', how='left').fillna(0)

# 4. Extract best parameters and build the final pipeline
final_model_params = {k: v for k, v in best_params_from_study.items() if k not in ['smoothing']}
final_smoothing = best_params_from_study.get('smoothing', 10)

final_preprocessor = ColumnTransformer([
    ('num', Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]), numerical_cols),
    ('cat', TargetEncoder(columns=categorical_cols, smoothing=final_smoothing), categorical_cols)
], remainder='drop', n_jobs=-1)

final_pipeline = Pipeline([
    ('preprocessor', final_preprocessor),
    ('regressor', xgb.XGBRegressor(**final_model_params, objective='reg:squarederror', random_state=42, n_jobs=-1))
])

# 5. Train on ALL available training data
final_pipeline.fit(X_train_final, y)
print("Final model trained successfully.")


print("\n--- Generating Final Kaggle Submission ---")
# Align test columns with train columns to ensure consistency
train_cols = X_train_final.columns
X_test_final = X_test_final.reindex(columns=train_cols, fill_value=0)

# Predict log purchase value
kaggle_log_preds = final_pipeline.predict(X_test_final)
kaggle_log_preds[kaggle_log_preds < 0] = 0

# Transform prediction back to original scale (inverse of log1p and scaling)
kaggle_value_preds = np.expm1(kaggle_log_preds) * 1e6

# Create submission file
submission_df = pd.DataFrame({'ID': df_test.index, 'purchaseValue': kaggle_value_preds})
submission_df.to_csv('submission_robust_model11.csv', index=False)
print("Submission file 'submission_robust_model.csv' created successfully.")
print(submission_df.head())

Loading and preparing data...


[I 2025-07-16 20:15:28,711] A new study created in memory with name: no-name-41972966-7cab-470e-b0d7-ba93600ca3b3



Identified 13 potential numerical features.
Identified 38 potential categorical features.

--- Tuning Regressor with Optuna (30 trials) using LEAK-FREE CV ---
  --- Fold 1/4 ---
    Fold 1 RMSE: 1.40970
  --- Fold 2/4 ---
    Fold 2 RMSE: 1.44570
  --- Fold 3/4 ---
    Fold 3 RMSE: 1.41829
  --- Fold 4/4 ---


[I 2025-07-16 20:15:50,868] Trial 0 finished with value: 1.4218815558037812 and parameters: {'n_estimators': 1200, 'learning_rate': 0.04195599971217711, 'max_depth': 9, 'subsample': 0.8538678095370361, 'colsample_bytree': 0.8724003816032452, 'gamma': 3.963598082497761, 'lambda': 5.264359622322329, 'alpha': 0.35225822524924344, 'smoothing': 19}. Best is trial 0 with value: 1.4218815558037812.


    Fold 4 RMSE: 1.41384
  Trial Mean RMSE: 1.42188

  --- Fold 1/4 ---
    Fold 1 RMSE: 1.33851
  --- Fold 2/4 ---
    Fold 2 RMSE: 1.34275
  --- Fold 3/4 ---
    Fold 3 RMSE: 1.34941
  --- Fold 4/4 ---


...

    Fold 4 RMSE: 1.33526
  Trial Mean RMSE: 1.34913

  --- Fold 1/4 ---
    Fold 1 RMSE: 1.32841
  --- Fold 2/4 ---
    Fold 2 RMSE: 1.34806
  --- Fold 3/4 ---
    Fold 3 RMSE: 1.35662
  --- Fold 4/4 ---


[I 2025-07-16 20:26:35,263] Trial 28 finished with value: 1.3390586935320696 and parameters: {'n_estimators': 1800, 'learning_rate': 0.017636993733221595, 'max_depth': 8, 'subsample': 0.7699437342408489, 'colsample_bytree': 0.8588921661771642, 'gamma': 4.6618070837453045, 'lambda': 1.6283736584791124, 'alpha': 2.91528470370736, 'smoothing': 6}. Best is trial 18 with value: 1.3264493830394408.


    Fold 4 RMSE: 1.32314
  Trial Mean RMSE: 1.33906

  --- Fold 1/4 ---
    Fold 1 RMSE: 1.36638
  --- Fold 2/4 ---
    Fold 2 RMSE: 1.37946
  --- Fold 3/4 ---
    Fold 3 RMSE: 1.36973
  --- Fold 4/4 ---


[I 2025-07-16 20:26:56,176] Trial 29 finished with value: 1.3644570477276092 and parameters: {'n_estimators': 1300, 'learning_rate': 0.02289427641126675, 'max_depth': 9, 'subsample': 0.8427056425411141, 'colsample_bytree': 0.7826022955412402, 'gamma': 4.174132445259895, 'lambda': 5.248461355615132, 'alpha': 4.2450241885580695, 'smoothing': 10}. Best is trial 18 with value: 1.3264493830394408.


    Fold 4 RMSE: 1.34226
  Trial Mean RMSE: 1.36446


Best trial:
  Value (RMSE): 1.32645
  Params: 
    n_estimators: 1900
    learning_rate: 0.011164649797353439
    max_depth: 6
    subsample: 0.7061146990552216
    colsample_bytree: 0.7821786922926288
    gamma: 4.560588523006475
    lambda: 2.388590274590411
    alpha: 3.6949247191996317
    smoothing: 15

--- Training Final Model with Best Parameters on ALL Data ---
Final model trained successfully.

--- Generating Final Kaggle Submission ---
Submission file 'submission_robust_model.csv' created successfully.
   ID  purchaseValue
0   0   60122.062500
1   1    2306.391602
2   2    2306.391602
3   3   16237.957031
4   4    2306.391602
